<a href="https://colab.research.google.com/github/cristinarainich/Sternberg_analysis/blob/main/iEEG_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import pylab as pl
import seaborn as sns
import re
from scipy import stats
import scipy.io

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
sns.set()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
working_path = Path('/content/drive/MyDrive/CoganLab/Neighbourhood')

In [34]:
ieeg = pd.read_csv(
    working_path.joinpath('ieeg_all.csv'),
    index_col=None
)

# Preprocessing

In [35]:
ieeg = ieeg.loc[:, ieeg.columns != 'Unnamed: 0'] # some extra column appeared while uploading data

In [36]:
ieeg.isnull().sum()

block                       0
ProbeType                   0
ProbeTypeName               0
StimlusCategory             0
ReactionTime                0
KeyCode                     0
Resp                        0
RespCorrect                 0
Omission                  160
stimulusAudioStart          0
stimulusAlignedTrigger      0
probeAudioStart             0
probeAlignedTrigger         0
TriggerValue                0
stimulusSounds_idx          0
stimulusSounds_name         0
ProbeCategory               0
probeSound_idx              0
probeSound_name             0
Trial                       0
Subject                     0
ListenCueTime             953
MaintenancePeriodTime     953
ProbeCueTime              953
dtype: int64

In [37]:
ieeg.shape

(5300, 24)

In [38]:
ieeg.Subject.unique()

array(['D23', 'D26', 'D27', 'D28', 'D29', 'D30', 'D31', 'D33', 'D34',
       'D35', 'D36', 'D38', 'D39', 'D41', 'D42', 'D47', 'D48', 'D49',
       'D53', 'D54', 'D55', 'D57', 'D59', 'D61', 'D63', 'D65', 'D68',
       'D69', 'D70', 'D71', 'D72', 'D73', 'D77'], dtype=object)

In [39]:
ieeg.columns

Index(['block', 'ProbeType', 'ProbeTypeName', 'StimlusCategory',
       'ReactionTime', 'KeyCode', 'Resp', 'RespCorrect', 'Omission',
       'stimulusAudioStart', 'stimulusAlignedTrigger', 'probeAudioStart',
       'probeAlignedTrigger', 'TriggerValue', 'stimulusSounds_idx',
       'stimulusSounds_name', 'ProbeCategory', 'probeSound_idx',
       'probeSound_name', 'Trial', 'Subject', 'ListenCueTime',
       'MaintenancePeriodTime', 'ProbeCueTime'],
      dtype='object')

In [40]:
ieeg[['stimulusSounds_name', 'StimlusCategory', 'probeSound_name', 'ProbeCategory']]

,stimulusSounds_name,StimlusCategory,probeSound_name,ProbeCategory
0,"[[array(['wash.wav'], dtype='<U8')]\n [array([...",Low Words,wash.wav,Low Words
1,"[[array(['debt.wav'], dtype='<U8')]\n [array([...",High Words,lair.wav,High Words
2,"[[array(['rihn.wav'], dtype='<U8')]\n [array([...",High Non-Words,paon.wav,High Non-Words
3,"[[array(['witch.wav'], dtype='<U9')]\n [array(...",High Words,wash.wav,Low Words
4,"[array(['youth.wav'], dtype='<U9'), array(['no...",Low Words,soar.wav,High Words
...,...,...,...,...
5295,"[[array(['witch.wav'], dtype='<U9')]\n [array(...",High Words,noun.wav,Low Words
5296,"[[array(['vern.wav'], dtype='<U8')]\n [array([...",Low Non-words,yuwf.wav,Low Non-words
5297,"[[array(['gear.wav'], dtype='<U8')]\n [array([...",High Words,tube.wav,Low Words
5298,"[[array(['lair.wav'], dtype='<U8')]\n [array([...",High Words,waif.wav,Low Words


In [41]:
ieeg_subset = ieeg[[
                 'Trial', 'Subject', 'block', 'ProbeType', 'probeSound_idx',
                 'ProbeTypeName', 'StimlusCategory', 'ReactionTime', 'KeyCode',
                 'Resp', 'RespCorrect', 'Omission', 'ProbeCategory',
                  'stimulusSounds_idx', 'stimulusSounds_name', 'probeSound_name'              
]]

In [42]:
ieeg_subset.head()

,Trial,Subject,block,ProbeType,probeSound_idx,ProbeTypeName,StimlusCategory,ReactionTime,KeyCode,Resp,RespCorrect,Omission,ProbeCategory,stimulusSounds_idx,stimulusSounds_name,probeSound_name
0,0,D23,1,2,29,in_sequence,Low Words,0.857518,37,Yes,1,Responded,Low Words,[[29 25 30]],"[[array(['wash.wav'], dtype='<U8')]\n [array([...",wash.wav
1,1,D23,1,2,5,in_sequence,High Words,1.339604,37,Yes,1,Responded,High Words,[[ 2 5 8 4 7 10 3]],"[[array(['debt.wav'], dtype='<U8')]\n [array([...",lair.wav
2,2,D23,1,2,16,in_sequence,High Non-Words,1.045657,37,Yes,1,Responded,High Non-Words,[[19 14 18 16 20 15 17]],"[[array(['rihn.wav'], dtype='<U8')]\n [array([...",paon.wav
3,3,D23,1,1,29,out_of_sequence,High Words,1.497001,39,No,1,Responded,Low Words,[[10 2 7 9 1 8 6 5 4]],"[[array(['witch.wav'], dtype='<U9')]\n [array(...",wash.wav
4,4,D23,1,1,9,out_of_sequence,Low Words,1.679783,39,No,1,Responded,High Words,"[30, 24, 25, 29, 21]","[array(['youth.wav'], dtype='<U9'), array(['no...",soar.wav


Checking for accuracy of subjects 

In [43]:
# check if any of the subjects is less than 75 % accurate
checkset = ieeg_subset.groupby(['Subject'])[['RespCorrect']].apply(
    lambda x: x.sum()/x.count()
)
(checkset < 0.75).any()

RespCorrect    True
dtype: bool

In [44]:
options_drop = checkset[checkset['RespCorrect'] < 0.75].index
checkset = checkset.reset_index()

print('The subjects with accuracy less than 75 %')
print(options_drop)
x = []
for i in checkset.Subject.values:
  for j in options_drop:
    if i == j:
      x.append(checkset.loc[checkset['Subject'] == i][['RespCorrect']].values[0][0])
print(x)  

The subjects with accuracy less than 75 %
Index(['D30', 'D33', 'D49', 'D54', 'D61', 'D65', 'D70', 'D77'], dtype='object', name='Subject')
[0.70625, 0.6310160427807486, 0.61875, 0.43125, 0.6125, 0.65, 0.71875, 0.725]


Cleaning 'No response' trials

In [45]:
ieeg_subset[ieeg_subset['Resp'] == 'No Response']

,Trial,Subject,block,ProbeType,probeSound_idx,ProbeTypeName,StimlusCategory,ReactionTime,KeyCode,Resp,RespCorrect,Omission,ProbeCategory,stimulusSounds_idx,stimulusSounds_name,probeSound_name
9,9,D23,1,2,39,in_sequence,Low Non-words,2.502478,[[37 39 32 27]],No Response,0,No Response,Low Non-words,[[32 37 31 39 40 35 33 38 36]],"[[array(['faez.wav'], dtype='<U8')]\n [array([...",yuwf.wav
19,19,D23,1,1,40,out_of_sequence,High Non-Words,2.502283,[[37 39 32 27]],No Response,0,No Response,Low Non-Words,[[17 19 18 13 15 12 16]],"[[array(['puwn.wav'], dtype='<U8')]\n [array([...",zerth.wav
26,26,D23,1,1,33,out_of_sequence,High Non-Words,2.502340,[[37 39 32 27]],No Response,0,No Response,Low Non-Words,[[13 18 14]],"[[array(['kawl.wav'], dtype='<U8')]\n [array([...",gawth.wav
32,32,D23,2,2,25,in_sequence,Low Words,2.502395,[[37 39 32 27]],No Response,0,No Response,Low Words,[[30 22 25 29 21 23 24 28 27]],"[[array(['youth.wav'], dtype='<U9')]\n [array(...",sure.wav
97,97,D23,4,1,36,out_of_sequence,High Non-Words,2.502652,[[37 39 32 27]],No Response,0,No Response,Low Non-Words,[[16 17 18]],"[[array(['paon.wav'], dtype='<U8')]\n [array([...",vern.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5217,77,D77,3,1,39,out_of_sequence,Low Non-words,2.501365,"[114, 115, 66, 10, 27]",No Response,0,No Response,Low Non-Words,[[38 36 34 32 35 37 31]],"[[array(['yong.wav'], dtype='<U8')]\n [array([...",yuwf.wav
5256,116,D77,4,1,40,out_of_sequence,Low Non-words,2.501420,"[114, 115, 66, 10, 27]",No Response,0,No Response,Low Non-Words,[[33 37 34]],"[[array(['gawth.wav'], dtype='<U9')]\n [array(...",zerth.wav
5260,120,D77,4,2,39,in_sequence,Low Non-words,2.501402,"[114, 115, 66, 10, 27]",No Response,0,No Response,Low Non-words,[[39 38 35]],"[[array(['yuwf.wav'], dtype='<U8')]\n [array([...",yuwf.wav
5278,138,D77,5,1,37,out_of_sequence,High Non-Words,2.501428,"[114, 115, 66, 10, 27]",No Response,0,No Response,Low Non-Words,[[17 14 16 12 19 15 13]],"[[array(['puwn.wav'], dtype='<U8')]\n [array([...",wehz.wav


In [46]:
ieeg_subset[ieeg_subset['Omission'] != 'Responded']
# let's keep these NaNs since all the other information is there     

,Trial,Subject,block,ProbeType,probeSound_idx,ProbeTypeName,StimlusCategory,ReactionTime,KeyCode,Resp,RespCorrect,Omission,ProbeCategory,stimulusSounds_idx,stimulusSounds_name,probeSound_name
9,9,D23,1,2,39,in_sequence,Low Non-words,2.502478,[[37 39 32 27]],No Response,0,No Response,Low Non-words,[[32 37 31 39 40 35 33 38 36]],"[[array(['faez.wav'], dtype='<U8')]\n [array([...",yuwf.wav
19,19,D23,1,1,40,out_of_sequence,High Non-Words,2.502283,[[37 39 32 27]],No Response,0,No Response,Low Non-Words,[[17 19 18 13 15 12 16]],"[[array(['puwn.wav'], dtype='<U8')]\n [array([...",zerth.wav
26,26,D23,1,1,33,out_of_sequence,High Non-Words,2.502340,[[37 39 32 27]],No Response,0,No Response,Low Non-Words,[[13 18 14]],"[[array(['kawl.wav'], dtype='<U8')]\n [array([...",gawth.wav
32,32,D23,2,2,25,in_sequence,Low Words,2.502395,[[37 39 32 27]],No Response,0,No Response,Low Words,[[30 22 25 29 21 23 24 28 27]],"[[array(['youth.wav'], dtype='<U9')]\n [array(...",sure.wav
97,97,D23,4,1,36,out_of_sequence,High Non-Words,2.502652,[[37 39 32 27]],No Response,0,No Response,Low Non-Words,[[16 17 18]],"[[array(['paon.wav'], dtype='<U8')]\n [array([...",vern.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5217,77,D77,3,1,39,out_of_sequence,Low Non-words,2.501365,"[114, 115, 66, 10, 27]",No Response,0,No Response,Low Non-Words,[[38 36 34 32 35 37 31]],"[[array(['yong.wav'], dtype='<U8')]\n [array([...",yuwf.wav
5256,116,D77,4,1,40,out_of_sequence,Low Non-words,2.501420,"[114, 115, 66, 10, 27]",No Response,0,No Response,Low Non-Words,[[33 37 34]],"[[array(['gawth.wav'], dtype='<U9')]\n [array(...",zerth.wav
5260,120,D77,4,2,39,in_sequence,Low Non-words,2.501402,"[114, 115, 66, 10, 27]",No Response,0,No Response,Low Non-words,[[39 38 35]],"[[array(['yuwf.wav'], dtype='<U8')]\n [array([...",yuwf.wav
5278,138,D77,5,1,37,out_of_sequence,High Non-Words,2.501428,"[114, 115, 66, 10, 27]",No Response,0,No Response,Low Non-Words,[[17 14 16 12 19 15 13]],"[[array(['puwn.wav'], dtype='<U8')]\n [array([...",wehz.wav


In [47]:
print(ieeg_subset[ieeg_subset['Omission'] != 'Responded'].Subject.unique())

['D23' 'D26' 'D27' 'D28' 'D30' 'D31' 'D33' 'D34' 'D35' 'D36' 'D38' 'D39'
 'D41' 'D42' 'D47' 'D48' 'D49' 'D53' 'D54' 'D55' 'D57' 'D59' 'D61' 'D63'
 'D65' 'D68' 'D69' 'D70' 'D71' 'D72' 'D73' 'D77']


Creating additional variables 

In [48]:
# types in ProbeCategory column differ in case
# here we make them lowrcase
new_probe = []
for i in ieeg_subset.ProbeCategory.values:
  new_probe.append(i.lower())
ieeg_subset['ProbeCategory'] = new_probe

<ipython-input-48-df1cb0339cf5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ieeg_subset['ProbeCategory'] = new_probe


In [49]:
# making values in df more 'handy'
whole_list = ieeg_subset['stimulusSounds_idx'].values
new_sound_idx = []
for i in whole_list:
  new_sound_idx.append(
      [int(s) for s in re.findall(r'\b\d+\b', i)]
  )
ieeg_subset['stimulusSounds_idx'] = new_sound_idx # substitute string type to list with numbers

<ipython-input-49-973a0a02d07e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ieeg_subset['stimulusSounds_idx'] = new_sound_idx # substitute string type to list with numbers


In [50]:
# checking the length of word lists presented
compare = []
for i in new_sound_idx:
  length = len(i)
  if length not in compare:
    compare.append(length)
print(compare)

[3, 7, 9, 5]


In [51]:
# calculating relative position of the word in the list
# pos for only in seqence trials: probe sound index and stimulus sound index
# beginning middle or end
positions = [] # for the new 'position' column in df
# 1 - beginning, 2 - middle, 3 - end
ind = ieeg_subset['probeSound_idx'].values
in_out = ieeg_subset['ProbeTypeName'].values
for i in range(0, len(ind)):
  if in_out[i] == 'in_sequence': # if it is in sequence
    if ind[i] in new_sound_idx[i]:
      # for length of the list 3
      if len(new_sound_idx[i]) == 3:
        if new_sound_idx[i].index(ind[i]) == 0:
          positions.append(1) # beginning 
        elif new_sound_idx[i].index(ind[i]) == 1:
          positions.append(2) # middle
        elif new_sound_idx[i].index(ind[i]) == 2:
          positions.append(3) # end
      
      # for length of the list 5
      ## revise!
      elif len(new_sound_idx[i]) == 5:
        if new_sound_idx[i].index(ind[i]) in [0, 1]:
          positions.append(1) # beginning 
        elif new_sound_idx[i].index(ind[i]) == 2:
          positions.append(2) # middle
        elif new_sound_idx[i].index(ind[i]) in [3, 4]:
          positions.append(3) # end

      # for length of the list 7
      elif len(new_sound_idx[i]) == 7:
        if new_sound_idx[i].index(ind[i]) in [0, 1]:
          positions.append(1) # beginning 
        elif new_sound_idx[i].index(ind[i]) in [2, 3, 4]:
          positions.append(2) # middle
        elif new_sound_idx[i].index(ind[i]) in [5, 6]:
          positions.append(3) # end
      
      # for length of the list 9
      elif len(new_sound_idx[i]) == 9:
        if new_sound_idx[i].index(ind[i]) in [0, 1, 2]:
          positions.append(1) # beginning 
        elif new_sound_idx[i].index(ind[i]) in [3, 4, 5]:
          positions.append(2) # middle
        elif new_sound_idx[i].index(ind[i]) in [6, 7, 8]:
          positions.append(3) # end
    else:
      print('Check', i)
      break  
  elif in_out[i] == 'out_of_sequence':
    positions.append(0) # if it is out of sequence it is not in the list
  else:
    print('Check', i)
    break

In [52]:
length = []
for i in ieeg_subset.stimulusSounds_idx:
  length.append(len(i))

In [53]:
ieeg_subset['Position'] = positions
ieeg_subset['Length'] = length

<ipython-input-53-b7de9913c462>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ieeg_subset['Position'] = positions
<ipython-input-53-b7de9913c462>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ieeg_subset['Length'] = length


In [54]:
lex = []
phon = []
for i in ieeg_subset.ProbeCategory.values:
  if 'non-words' in i: # non - words = 0, words = 0
    lex.append(0)
  elif 'words' in i:
    lex.append(1)
for i in ieeg_subset.ProbeCategory.values:
  if 'low' in i: # high = 0 low = 1 
    phon.append(1)
  elif 'high' in i:
    phon.append(0)
print(len(lex))
print(len(phon))

5300
5300


In [55]:
ieeg_subset['Lex'] = lex
ieeg_subset['Phono'] = phon

<ipython-input-55-aefbe34290ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ieeg_subset['Lex'] = lex
<ipython-input-55-aefbe34290ea>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ieeg_subset['Phono'] = phon


In [56]:
ieeg_subset.head()

,Trial,Subject,block,ProbeType,probeSound_idx,ProbeTypeName,StimlusCategory,ReactionTime,KeyCode,Resp,RespCorrect,Omission,ProbeCategory,stimulusSounds_idx,stimulusSounds_name,probeSound_name,Position,Length,Lex,Phono
0,0,D23,1,2,29,in_sequence,Low Words,0.857518,37,Yes,1,Responded,low words,"[29, 25, 30]","[[array(['wash.wav'], dtype='<U8')]\n [array([...",wash.wav,1,3,1,1
1,1,D23,1,2,5,in_sequence,High Words,1.339604,37,Yes,1,Responded,high words,"[2, 5, 8, 4, 7, 10, 3]","[[array(['debt.wav'], dtype='<U8')]\n [array([...",lair.wav,1,7,1,0
2,2,D23,1,2,16,in_sequence,High Non-Words,1.045657,37,Yes,1,Responded,high non-words,"[19, 14, 18, 16, 20, 15, 17]","[[array(['rihn.wav'], dtype='<U8')]\n [array([...",paon.wav,2,7,0,0
3,3,D23,1,1,29,out_of_sequence,High Words,1.497001,39,No,1,Responded,low words,"[10, 2, 7, 9, 1, 8, 6, 5, 4]","[[array(['witch.wav'], dtype='<U9')]\n [array(...",wash.wav,0,9,1,1
4,4,D23,1,1,9,out_of_sequence,Low Words,1.679783,39,No,1,Responded,high words,"[30, 24, 25, 29, 21]","[array(['youth.wav'], dtype='<U9'), array(['no...",soar.wav,0,5,1,0


## Plotting the accuracy across conditions

In [57]:
ieeg_no_outliers.head()

NameError: ignored

In [ ]:
only_match = ieeg_no_outliers[ieeg_no_outliers['ProbeType'] == 2]
only_mismatch = ieeg_no_outliers[ieeg_no_outliers['ProbeType'] == 1]

In [ ]:
ieeg_no_outliers.groupby(['ProbeType'])[['RespCorrect']].mean()

In [ ]:
ss = ieeg_no_outliers[ieeg_no_outliers['ProbeType'] == 1][['ProbeCategory', 'StimlusCategory']]

In [ ]:
new_p = []
new_s = []
for i in ss.ProbeCategory.values:
  new_p.append(i.lower())
for i in ss.StimlusCategory.values:
  new_s.append(i.lower())
print(len(new_p))
print(len(new_s))

In [ ]:
ss['ProbeCategory'] = new_p
ss['StimlusCategory'] = new_s

In [ ]:
index = 0
check = []
for i in range(0, ss.shape[0]):
  if 'non-words' in new_p[i]:
    if 'non-words' in new_s[i]:
      index += 1 
    else:
      index += 1
      check.append(i)
  else:
    if 'non-words' in new_s[i]:
      index += 1
      check.append(i)
    else:
      index += 1
print(index)
print(check)

**General plots**

In [ ]:
ieeg_no_outliers.groupby(['ProbeCategory'])['RespCorrect'].mean().plot(
    legend=True, title = 'Mean Acc across conditions'
    );

In [ ]:
ieeg_no_outliers.groupby(['Subject'])['RespCorrect'].mean().plot(
    legend=True, title = 'Mean Acc across subjects'
    );

In [ ]:
# ALL TOGETHER
# percentages of correct responses for different conditions
# acc = correct trials in this cond / all trials in this cond
pl.figure()
subset = ieeg_no_outliers.groupby(['ProbeCategory', 'Length'])[['RespCorrect']].mean().reset_index()
for i in subset.ProbeCategory.unique():
  x = subset[subset['ProbeCategory'] == i]
  pl.plot(x.Length, x.RespCorrect)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('Acc')
pl.legend(subset.ProbeCategory.unique())
pl.title('Acc across conditions and line length');  

In [ ]:
# WHEN MATCH
# percentages of correct responses for different conditions
# acc = correct trials in this cond / all trials in this cond
pl.figure()
subset = only_match.groupby(['ProbeCategory', 'Length'])[['RespCorrect']].mean().reset_index()
for i in subset.ProbeCategory.unique():
  x = subset[subset['ProbeCategory'] == i]
  pl.plot(x.Length, x.RespCorrect)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('Acc')
pl.legend(subset.ProbeCategory.unique())
pl.title('Acc across conditions and line length only match');  

In [ ]:
only_match.groupby(['ProbeCategory'])[['RespCorrect']].mean()

In [ ]:
only_mismatch.groupby(['ProbeCategory'])[['RespCorrect']].mean()

In [ ]:
only_match.groupby(['Phono'])[['RespCorrect']].mean()

In [ ]:
only_mismatch.groupby(['Phono'])[['RespCorrect']].mean()

In [ ]:
# WHEN MISMATCH
# percentages of correct responses for different conditions
# acc = correct trials in this cond / all trials in this cond
pl.figure()
subset = only_mismatch.groupby(['ProbeCategory', 'Length'])[['RespCorrect']].mean().reset_index()
for i in subset.ProbeCategory.unique():
  x = subset[subset['ProbeCategory'] == i]
  pl.plot(x.Length, x.RespCorrect)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('Acc')
pl.legend(subset.ProbeCategory.unique())
pl.title('Acc across conditions and line length only mismatch');  

In [ ]:
# WHEN MISMATCH
# percentages of correct responses for different conditions
# acc = correct trials in this cond / all trials in this cond
pl.figure()
subset = only_mismatch.groupby(['StimlusCategory', 'Length'])[['RespCorrect']].mean().reset_index()
for i in subset.StimlusCategory.unique():
  x = subset[subset['StimlusCategory'] == i]
  pl.plot(x.Length, x.RespCorrect)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('Acc')
pl.legend(subset.StimlusCategory.unique())
pl.title('Acc across conditions and line length only mismatch');  

In [ ]:
# W VS. NW FOR BOTH MATCH AND MISMATCH
pl.figure()
words_nonwords = ieeg_no_outliers.groupby(['Lex', 'Length'])[['RespCorrect']].mean(). reset_index()
for i in words_nonwords.Lex.unique():
  x = words_nonwords[words_nonwords['Lex'] == i]
  pl.plot(x.Length, x.RespCorrect)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('Acc')
pl.legend(words_nonwords.Lex.unique())
pl.title('Words (1) vs. non-words (0)');  

In [ ]:
# W VS. NW WHEN MATCH
pl.figure()
words_nonwords = only_match.groupby(['Lex', 'Length'])[['RespCorrect']].mean(). reset_index()
for i in words_nonwords.Lex.unique():
  x = words_nonwords[words_nonwords['Lex'] == i]
  pl.plot(x.Length, x.RespCorrect)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('Acc')
pl.legend(words_nonwords.Lex.unique())
pl.title('Words (1) vs. non-words (0) only match'); 

In [ ]:
# W VS. NW WHEN MISMATCH
pl.figure()
words_nonwords = only_mismatch.groupby(['Lex', 'Length'])[['RespCorrect']].mean(). reset_index()
for i in words_nonwords.Lex.unique():
  x = words_nonwords[words_nonwords['Lex'] == i]
  pl.plot(x.Length, x.RespCorrect)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('Acc')
pl.legend(words_nonwords.Lex.unique())
pl.title('Words (1) vs. non-words (0) only mismatch'); 

In [ ]:
# H VS LOW FOR BOTH MATCH AND MISMATCH
pl.figure()
high_low = ieeg_no_outliers.groupby(['Phono', 'Length'])[['RespCorrect']].mean().reset_index()
for i in high_low.Phono.unique():
  x = high_low[high_low['Phono'] == i]
  pl.plot(x.Length, x.RespCorrect)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('Acc')
pl.legend(high_low.Phono.unique())
pl.title('High vs. Low'); 

In [ ]:
# H VS LOW WHEN MATCH
pl.figure()
high_low = only_match.groupby(['Phono', 'Length'])[['RespCorrect']].mean().reset_index()
for i in high_low.Phono.unique():
  x = high_low[high_low['Phono'] == i]
  pl.plot(x.Length, x.RespCorrect)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('Acc')
pl.legend(high_low.Phono.unique())
pl.title('High vs. Low only match'); 

In [ ]:
# H VS LOW WHEN MISMATCH
pl.figure()
high_low = only_mismatch.groupby(['Phono', 'Length'])[['RespCorrect']].mean().reset_index()
for i in high_low.Phono.unique():
  x = high_low[high_low['Phono'] == i]
  pl.plot(x.Length, x.RespCorrect)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('Acc')
pl.legend(high_low.Phono.unique())
pl.title('High vs. Low only mismatch'); 

In [ ]:
# probe type 2 = in seq, 1 = out of seq
probe_words = ieeg_no_outliers.groupby(['ProbeType', 'Lex', 'Length'])[['RespCorrect']].mean().reset_index()
for i in probe_words.ProbeType.unique():
  for j in probe_words.Lex.unique():
    x = probe_words[(probe_words['ProbeType'] == i) & (probe_words['Lex'] == j)]
    pl.plot(x.Length, x.RespCorrect, label = ('Probe', i, 'Lex', j))
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('Acc')
pl.legend()
pl.title('Words/Non-words and Probe/No probe');

In [ ]:
probe_neigh = ieeg_no_outliers.groupby(['ProbeType', 'Phono', 'Length'])[['RespCorrect']].mean().reset_index()
for i in probe_neigh.ProbeType.unique():
  for j in probe_neigh.Phono.unique():
    x = probe_neigh[(probe_neigh['ProbeType'] == i) & (probe_neigh['Phono'] == j)]
    pl.plot(x.Length, x.RespCorrect, label = ('Probe', i, 'Phono', j))
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('Acc')
pl.legend()
pl.title('High/Low and Probe(2)/No probe(1)');

In [ ]:
subset = ieeg_no_outliers.groupby(['Position', 'Length'])[['RespCorrect']].mean().reset_index()
for i in subset.Position.unique():
  x = subset[subset['Position'] == i]
  pl.plot(x.Length, x.RespCorrect)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('Acc')
pl.legend(subset.Position.unique())
pl.title('Position vs. Length');

In [ ]:
subset = only_match.groupby(['Position', 'Length'])[['RespCorrect']].mean().reset_index()
for i in subset.Position.unique():
  x = subset[subset['Position'] == i]
  pl.plot(x.Length, x.RespCorrect)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('Acc')
pl.legend(subset.Position.unique())
pl.title('Position vs. Length only match');

In [ ]:
pos_words = ieeg_no_outliers.groupby(['Position', 'Lex', 'Length'])[['RespCorrect']].mean().reset_index()
for i in pos_words.Position.unique():
  for j in pos_words.Lex.unique():
    x = pos_words[(pos_words['Position'] == i) & (pos_words['Lex'] == j)]
    if j == 0:
      pl.plot(x.Length, x.RespCorrect, '--', label = ('Pos', i, 'Lex', j))
    elif j == 1:
      pl.plot(x.Length, x.RespCorrect, label = ('Pos', i, 'Lex', j))
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('Acc')
pl.legend()
pl.title('Position vs. W/NW vs. Length');

## Plotting accuracy distributions per subject

In [ ]:
ieeg_no_outliers.head()

In [ ]:
for i in ieeg_no_outliers.Subject.unique():
  subset = ieeg_no_outliers[ieeg_no_outliers['Subject'] == i]
  pl.figure()
  sns.barplot(x = 'Lex', y = 'RespCorrect', data = subset,
              estimator = lambda x: sum(x)/len(x));
  pl.title('Non-words vs. words for ' + subset.Subject.unique());

Paired t-test

In [ ]:
sample = ieeg_no_outliers[ieeg_no_outliers['Subject'] == 'D27']
sample = sample[[
        'Lex', 'RespCorrect'
]]

In [ ]:
x = sample[sample['Lex'] == 1]
y = sample[sample['Lex'] == 0]
stats.ttest_ind(x, y, equal_var = False)

Plots

In [ ]:
for i in ieeg_no_outliers.Subject.unique():
  subset = ieeg_no_outliers[ieeg_no_outliers['Subject'] == i]
  pl.figure()
  sns.barplot(x = 'Phono', y = 'RespCorrect', data = subset,
              estimator = lambda x: sum(x)/len(x));
  pl.title('Low vs. high for ' + subset.Subject.unique());

Check for significance

In [ ]:
sample = ieeg_no_outliers[ieeg_no_outliers['Subject'] == 'D36']
sample = sample[[
                 'Phono', 'RespCorrect'
]]

In [ ]:
x = sample[sample['Phono'] == 1]
y = sample[sample['Phono'] == 0]
stats.ttest_ind(x, y, equal_var = False)

Plots

In [ ]:
for i in ieeg_no_outliers.Subject.unique():
  subset = ieeg_no_outliers[ieeg_no_outliers['Subject'] == i]
  pl.figure();
  sns.barplot(x = 'ProbeCategory', y = 'RespCorrect', data = subset, estimator = lambda x: sum(x)/len(x));
  pl.title(subset.Subject.unique());

In [ ]:
sample = ieeg_no_outliers[ieeg_no_outliers['Subject'] == 'D71']
sample = sample[[
                 'ProbeCategory', 'RespCorrect', 'ProbeType'
]]

In [ ]:
x = sample[sample['ProbeCategory'] == 'low words']['RespCorrect'].values
y = sample[sample['ProbeCategory'] == 'low non-words']['RespCorrect'].values
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
x = sample[sample['ProbeCategory'] == 'high words']['RespCorrect'].values
y = sample[sample['ProbeCategory'] == 'high non-words']['RespCorrect'].values
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
x = sample[sample['ProbeCategory'] == 'high words']['RespCorrect'].values
y = sample[sample['ProbeCategory'] == 'low words']['RespCorrect'].values
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
x = sample[sample['ProbeCategory'] == 'high non-words']['RespCorrect'].values
y = sample[sample['ProbeCategory'] == 'low non-words']['RespCorrect'].values
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
x = sample[(sample['ProbeCategory'] == 'low words') & (sample['ProbeType'] == 2)]['RespCorrect'].values
y = sample[(sample['ProbeCategory'] == 'low non-words') & (sample['ProbeType'] == 2)]['RespCorrect'].values
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
x = sample[(sample['ProbeCategory'] == 'low words') & (sample['ProbeType'] == 1)]['RespCorrect'].values
y = sample[(sample['ProbeCategory'] == 'low non-words') & (sample['ProbeType'] == 1)]['RespCorrect'].values
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
x = sample[(sample['ProbeCategory'] == 'high words') & (sample['ProbeType'] == 2)]['RespCorrect'].values
y = sample[(sample['ProbeCategory'] == 'high non-words') & (sample['ProbeType'] == 2)]['RespCorrect'].values
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
x = sample[(sample['ProbeCategory'] == 'high words') & (sample['ProbeType'] == 1)]['RespCorrect'].values
y = sample[(sample['ProbeCategory'] == 'high non-words') & (sample['ProbeType'] == 1)]['RespCorrect'].values
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
x = sample[(sample['ProbeCategory'] == 'low non-words') & (sample['ProbeType'] == 2)]['RespCorrect'].values
y = sample[(sample['ProbeCategory'] == 'high non-words') & (sample['ProbeType'] == 2)]['RespCorrect'].values
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
x = sample[(sample['ProbeCategory'] == 'low non-words') & (sample['ProbeType'] == 1)]['RespCorrect'].values
y = sample[(sample['ProbeCategory'] == 'high non-words') & (sample['ProbeType'] == 1)]['RespCorrect'].values
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
x = sample[(sample['ProbeCategory'] == 'low words') & (sample['ProbeType'] == 2)]['RespCorrect'].values
y = sample[(sample['ProbeCategory'] == 'high words') & (sample['ProbeType'] == 2)]['RespCorrect'].values
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
x = sample[(sample['ProbeCategory'] == 'low words') & (sample['ProbeType'] == 1)]['RespCorrect'].values
y = sample[(sample['ProbeCategory'] == 'high words') & (sample['ProbeType'] == 1)]['RespCorrect'].values
stats.ttest_ind(x, y, equal_var = False)

## Plotting the RT across conditions

In [ ]:
subset = ieeg_no_outliers.groupby(['ProbeCategory', 'Length'])[['stdRT']].mean().reset_index()
pl.figure()
for i in subset.ProbeCategory.unique():
  x = subset[subset['ProbeCategory'] == i]
  pl.plot(x.Length, x.stdRT)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('std RT')
pl.legend(subset.ProbeCategory.unique())
pl.title('RT across conditions and line length'); 

In [ ]:
subset = only_match.groupby(['ProbeCategory', 'Length'])[['stdRT']].mean().reset_index()
pl.figure()
for i in subset.ProbeCategory.unique():
  x = subset[subset['ProbeCategory'] == i]
  pl.plot(x.Length, x.stdRT)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('std RT')
pl.legend(subset.ProbeCategory.unique())
pl.title('RT across conditions and line length only match'); 

In [ ]:
subset = only_mismatch.groupby(['ProbeCategory', 'Length'])[['stdRT']].mean().reset_index()
pl.figure()
for i in subset.ProbeCategory.unique():
  x = subset[subset['ProbeCategory'] == i]
  pl.plot(x.Length, x.stdRT)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('std RT')
pl.legend(subset.ProbeCategory.unique())
pl.title('RT across conditions and line length only mismatch'); 

In [ ]:
subset = only_mismatch.groupby(['StimlusCategory', 'Length'])[['stdRT']].mean().reset_index()
pl.figure()
for i in subset.StimlusCategory.unique():
  x = subset[subset['StimlusCategory'] == i]
  pl.plot(x.Length, x.stdRT)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('std RT')
pl.legend(subset.StimlusCategory.unique())
pl.title('RT across conditions and line length only mismatch'); 

In [ ]:
pl.figure()
words_nonwords = ieeg_no_outliers.groupby(['Lex', 'Length'])[['stdRT']].mean().reset_index()
for i in words_nonwords.Lex.unique():
  x = words_nonwords[words_nonwords['Lex'] == i]
  pl.plot(x.Length, x.stdRT)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('RT')
pl.legend(words_nonwords.Lex.unique())
pl.title('Words vs. non-words');  

In [ ]:
pl.figure()
words_nonwords = only_match.groupby(['Lex', 'Length'])[['stdRT']].mean().reset_index()
for i in words_nonwords.Lex.unique():
  x = words_nonwords[words_nonwords['Lex'] == i]
  pl.plot(x.Length, x.stdRT)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('RT')
pl.legend(words_nonwords.Lex.unique())
pl.title('Words vs. non-words only match');  

In [ ]:
pl.figure()
words_nonwords = only_mismatch.groupby(['Lex', 'Length'])[['stdRT']].mean().reset_index()
for i in words_nonwords.Lex.unique():
  x = words_nonwords[words_nonwords['Lex'] == i]
  pl.plot(x.Length, x.stdRT)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('RT')
pl.legend(words_nonwords.Lex.unique())
pl.title('Words vs. non-words only mismatch');  

In [ ]:
pl.figure()
high_low = ieeg_no_outliers.groupby(['Phono', 'Length'])[['stdRT']].mean().reset_index()
for i in high_low.Phono.unique():
  x = high_low[high_low['Phono'] == i]
  pl.plot(x.Length, x.stdRT)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('RT')
pl.legend(high_low.Phono.unique())
pl.title('High vs. Low');  

In [ ]:
pl.figure()
high_low = only_match.groupby(['Phono', 'Length'])[['stdRT']].mean().reset_index()
for i in high_low.Phono.unique():
  x = high_low[high_low['Phono'] == i]
  pl.plot(x.Length, x.stdRT)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('RT')
pl.legend(high_low.Phono.unique())
pl.title('High vs. Low only match');

In [ ]:
pl.figure()
high_low = only_mismatch.groupby(['Phono', 'Length'])[['stdRT']].mean().reset_index()
for i in high_low.Phono.unique():
  x = high_low[high_low['Phono'] == i]
  pl.plot(x.Length, x.stdRT)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('RT')
pl.legend(high_low.Phono.unique())
pl.title('High vs. Low only mismatch');

## Individual distributions of RT per subject

In [ ]:
for i in ieeg_no_outliers.Subject.unique():
  subset = ieeg_no_outliers[ieeg_no_outliers['Subject'] == i]
  pl.figure()
  sns.barplot(x = 'ProbeCategory', y = 'stdRT', data = subset,
      estimator = lambda x: sum(x)/len(x))
  pl.title(subset.Subject.unique())

In [ ]:
for i in ieeg_no_outliers.Subject.unique():
  subset = ieeg_no_outliers[ieeg_no_outliers['Subject'] == i]
  pl.figure();
  sns.barplot(x = 'Lex', y = 'stdRT', data = subset, 
              estimator = lambda x: sum(x)/len(x));
  pl.title('NW vs. W for ' + subset.Subject.unique());

In [ ]:
sample = ieeg_no_outliers[ieeg_no_outliers['Subject'] == 'D27']
sample = sample[[
                 'Lex', 'ReactionTime', 'ProbeType'
]]

In [ ]:
x = sample[(sample['Lex'] == 1) & (sample['ProbeType'] == 2)]
y = sample[(sample['Lex'] == 0) & (sample['ProbeType'] == 2)]
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
sample = ieeg_no_outliers[ieeg_no_outliers['Subject'] == 'D28']
sample = sample[[
                 'ProbeCategory', 'ProbeType', 'ReactionTime'
]]

In [ ]:
x = sample[sample['ProbeCategory'] == 'low words']['ReactionTime'].values
y = sample[sample['ProbeCategory'] == 'low non-words']['ReactionTime'].values
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
x = sample[sample['ProbeCategory'] == 'high words']['ReactionTime'].values
y = sample[sample['ProbeCategory'] == 'high non-words']['ReactionTime'].values
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
x = sample[sample['ProbeCategory'] == 'low words']['ReactionTime'].values
y = sample[sample['ProbeCategory'] == 'high words']['ReactionTime'].values
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
x = sample[sample['ProbeCategory'] == 'low non-words']['ReactionTime'].values
y = sample[sample['ProbeCategory'] == 'high non-words']['ReactionTime'].values
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
x = sample[(sample['ProbeCategory'] == 'low words') & (sample['ProbeType'] == 2)]['ReactionTime'].values
y = sample[(sample['ProbeCategory'] == 'low non-words') & (sample['ProbeType'] == 2)]['ReactionTime'].values
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
x = sample[(sample['ProbeCategory'] == 'low words') & (sample['ProbeType'] == 1)]['ReactionTime'].values
y = sample[(sample['ProbeCategory'] == 'low non-words') & (sample['ProbeType'] == 1)]['ReactionTime'].values
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
x = sample[(sample['ProbeCategory'] == 'high words') & (sample['ProbeType'] == 2)]['ReactionTime'].values
y = sample[(sample['ProbeCategory'] == 'high non-words') & (sample['ProbeType'] == 2)]['ReactionTime'].values
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
x = sample[(sample['ProbeCategory'] == 'high words') & (sample['ProbeType'] == 1)]['ReactionTime'].values
y = sample[(sample['ProbeCategory'] == 'high non-words') & (sample['ProbeType'] == 1)]['ReactionTime'].values
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
x = sample[(sample['ProbeCategory'] == 'low non-words') & (sample['ProbeType'] == 2)]['ReactionTime'].values
y = sample[(sample['ProbeCategory'] == 'high non-words') & (sample['ProbeType'] == 2)]['ReactionTime'].values
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
x = sample[(sample['ProbeCategory'] == 'low non-words') & (sample['ProbeType'] == 1)]['ReactionTime'].values
y = sample[(sample['ProbeCategory'] == 'high non-words') & (sample['ProbeType'] == 1)]['ReactionTime'].values
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
x = sample[(sample['ProbeCategory'] == 'low words') & (sample['ProbeType'] == 2)]['ReactionTime'].values
y = sample[(sample['ProbeCategory'] == 'high words') & (sample['ProbeType'] == 2)]['ReactionTime'].values
stats.ttest_ind(x, y, equal_var = False)

In [ ]:
x = sample[(sample['ProbeCategory'] == 'low words') & (sample['ProbeType'] == 1)]['ReactionTime'].values
y = sample[(sample['ProbeCategory'] == 'high words') & (sample['ProbeType'] == 1)]['ReactionTime'].values
stats.ttest_ind(x, y, equal_var = False)

Plotting

In [ ]:
for i in ieeg_no_outliers.Subject.unique():
  subset = ieeg_no_outliers[ieeg_no_outliers['Subject'] == i]
  pl.figure();
  sns.barplot(x = 'Phono', y = 'stdRT', data = subset, 
              estimator = lambda x: sum(x)/len(x));
  pl.title('Low vs. High for ' + subset.Subject.unique());

In [ ]:
sample = ieeg_no_outliers[ieeg_no_outliers['Subject'] == 'D41']
sample = sample[[
                 'Phono', 'ReactionTime', 'ProbeType'
]]

In [ ]:
x = sample[(sample['Phono'] == 1) & (sample['ProbeType'] == 2)]
y = sample[(sample['Phono'] == 0) & (sample['ProbeType'] == 2)]
stats.ttest_ind(x, y, equal_var = False)

## **Only for correct responses**

In [ ]:
correct = ieeg_no_outliers[ieeg_no_outliers['RespCorrect'] == 1]
incorrect = ieeg_no_outliers[ieeg_no_outliers['RespCorrect'] == 0]

In [ ]:
correct.head()

In [ ]:
subset = correct.groupby(['ProbeCategory', 'Length'])[['stdRT']].mean().reset_index()
pl.figure()
for i in subset.ProbeCategory.unique():
  x = subset[subset['ProbeCategory'] == i]
  pl.plot(x.Length, x.stdRT)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('RT')
pl.legend(subset.ProbeCategory.unique())
pl.title('stand RT across conditions and line length only correct'); 

In [ ]:
subset = incorrect.groupby(['ProbeCategory', 'Length'])[['stdRT']].mean().reset_index()
pl.figure()
for i in subset.ProbeCategory.unique():
  x = subset[subset['ProbeCategory'] == i]
  pl.plot(x.Length, x.stdRT)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('RT')
pl.legend(subset.ProbeCategory.unique())
pl.title('stand RT across conditions and line length only incorrect');  

In [ ]:
pl.figure()
words_nonwords = correct.groupby(['Lex', 'Length'])[['stdRT']].mean().reset_index()
for i in words_nonwords.Lex.unique():
  x = words_nonwords[words_nonwords['Lex'] == i]
  pl.plot(x.Length, x.stdRT)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('RT')
pl.legend(words_nonwords.Lex.unique())
pl.title('Words vs. non-words only correct');  

In [ ]:
pl.figure()
words_nonwords = incorrect.groupby(['Lex', 'Length'])[['stdRT']].mean().reset_index()
for i in words_nonwords.Lex.unique():
  x = words_nonwords[words_nonwords['Lex'] == i]
  pl.plot(x.Length, x.stdRT)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('RT')
pl.legend(words_nonwords.Lex.unique())
pl.title('Words vs. non-words only incorrect'); 

In [ ]:
pl.figure()
words_nonwords = correct.groupby(['Phono', 'Length'])[['stdRT']].mean().reset_index()
for i in words_nonwords.Phono.unique():
  x = words_nonwords[words_nonwords['Phono'] == i]
  pl.plot(x.Length, x.stdRT)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('RT')
pl.legend(words_nonwords.Phono.unique())
pl.title('High vs. low only correct'); 

In [ ]:
pl.figure()
words_nonwords = incorrect.groupby(['Phono', 'Length'])[['stdRT']].mean().reset_index()
for i in words_nonwords.Phono.unique():
  x = words_nonwords[words_nonwords['Phono'] == i]
  pl.plot(x.Length, x.stdRT)
pl.xticks(x.Length)
pl.xlabel('List length')
pl.ylabel('RT')
pl.legend(words_nonwords.Phono.unique())
pl.title('High vs. low only incorrect'); 

## Saving result

In [58]:
# saving result
ieeg_subset.to_csv(
    Path(working_path).joinpath('ieeg_subset.csv')
)

In [59]:
ieeg_subset.columns

Index(['Trial', 'Subject', 'block', 'ProbeType', 'probeSound_idx',
       'ProbeTypeName', 'StimlusCategory', 'ReactionTime', 'KeyCode', 'Resp',
       'RespCorrect', 'Omission', 'ProbeCategory', 'stimulusSounds_idx',
       'stimulusSounds_name', 'probeSound_name', 'Position', 'Length', 'Lex',
       'Phono'],
      dtype='object')